In [1]:

import os
import sys
import random
import logging
import rasterio
import argparse
import numpy as np
from tqdm import tqdm
from os.path import dirname as up

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

path_cur = os.path.abspath(os.getcwd())

sys.path.append(path_cur)
from unet import UNet
from vims_dataloader import GenDEBRIS, bands_mean, bands_std

sys.path.append(os.path.join(up(up(path_cur)), 'utils'))

from vims_metrics import Evaluation, confusion_matrix
from vims_assets import labels
from pathlib import Path

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

root_path = up(up(path_cur))

logging.basicConfig(filename=os.path.join(root_path, 'logs','evaluating_unet.log'), filemode='a',level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')
logging.info('*'*10)



In [2]:
path_cur

'/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/semantic_segmentation/unet'

In [3]:
def main(options):
    # Transformations
    
    transform_test = transforms.Compose([transforms.ToTensor()])
    standardization = transforms.Normalize(bands_mean, bands_std)
    
    # Construct Data loader

    dataset_test = GenDEBRIS('test', transform=transform_test, standardization = standardization, agg_to_water = options['agg_to_water'])

    test_loader = DataLoader(   dataset_test, 
                                batch_size = options['batch'], 
                                shuffle = False)
    
    global labels
    # Aggregate Distribution Mixed Water, Wakes, Cloud Shadows, Waves with Marine Water

#     if options['agg_to_water']:
#         labels = labels[:-4] # Drop Mixed Water, Wakes, Cloud Shadows, Waves

#     # Use gpu or cpu
#     if torch.cuda.is_available():
#         device = torch.device("cuda")
#     else:
#         device = torch.device("cpu")


    device = torch.device("cpu")
        
    model = UNet(input_bands = options['input_channels'], 
                 output_classes = options['output_channels'], 
                 hidden_channels = options['hidden_channels'])

    model.to(device)

    # Load model from specific epoch to continue the training or start the evaluation
    model_file = options['model_path']
    logging.info('Loading model files from folder: %s' % model_file)

    checkpoint = torch.load(model_file, map_location = device)
    model.load_state_dict(checkpoint)

    del checkpoint  # dereference
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    model.eval()

    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        for (image, target) in tqdm(test_loader, desc="testing"):

            image = image.to(device)
            target = target.to(device)

            logits = model(image)

            # Accuracy metrics only on annotated pixels
            logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
            logits = logits.reshape((-1,options['output_channels']))
            target = target.reshape(-1)
            mask = target != -1
            logits = logits[mask]
            target = target[mask]
            
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            target = target.cpu().numpy()
            
            y_predicted += probs.argmax(1).tolist()
            y_true += target.tolist()
        
        ####################################################################
        # Save Scores to the .log file                                     #
        ####################################################################
        acc = Evaluation(y_predicted, y_true)
        logging.info("\n")
        logging.info("STATISTICS: \n")
        logging.info("Evaluation: " + str(acc))
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        conf_mat = confusion_matrix(y_true, y_predicted, labels)
        
        logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        
        if options['predict_masks']:
            
            path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test')
            ROIs = [file for file in os.listdir(path) if file.split('.')[-1]=='tif']

            impute_nan = np.tile(bands_mean, (256,256,1))
                        
            for roi in tqdm(ROIs):
            
#                 roi_folder = '_'.join(['S2'] + roi.split('_')[:-1])             # Get Folder Name
#                 roi_name = '_'.join(['S2'] + roi.split('_'))                    # Get File Name
#                 roi_file = os.path.join(path, roi_folder,roi_name + '.tif')     # Get File path
                
                roi_file = os.path.join(path, roi)
                os.makedirs(options['gen_masks_path'], exist_ok=True)
            
                output_image = os.path.join(options['gen_masks_path'], os.path.basename(roi_file).split('.tif')[0] + '_unet.tif')
            
                # Read metadata of the initial image
                with rasterio.open(roi_file, mode ='r') as src:
                    tags = src.tags().copy()
                    meta = src.meta
                    image = src.read()
                    image = np.moveaxis(image, (0, 1, 2), (2, 0, 1))
                    dtype = src.read(1).dtype
            
                # Update meta to reflect the number of layers
                meta.update(count = 1)
            
                # Write it
                with rasterio.open(output_image, 'w', **meta) as dst:
                    
                    # Preprocessing before prediction
                    nan_mask = np.isnan(image)
                    image[nan_mask] = impute_nan[nan_mask]
            
                    image = transform_test(image)
                    
                    image = standardization(image)
                    
                    # Image to Cuda if exist
                    image = image.to(device)
            
                    # Predictions
                    logits = model(image.unsqueeze(0))
            
                    probs = torch.nn.functional.softmax(logits.detach(), dim=1).cpu().numpy()
            
                    probs = probs.argmax(1).squeeze()+1
                    
                    # Write the mask with georeference
                    dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
                    dst.update_tags(**tags)



In [6]:
model_path = os.path.join(path_cur, 'trained_models', '200', 'model.pth')
gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')


options = {'agg_to_water': True, 'batch': 5, 
          'input_channels': 4, 'output_channels': 4, 'hidden_channels': 16,
          'model_path': model_path, 'predict_masks': True, 'gen_masks_path':gen_masks_path}

In [7]:
main(options)

testing: 100%|██████████| 498/498 [00:40<00:00, 12.18it/s]


Evaluation: {'macroPrec': 0.549332248723822, 'microPrec': 0.7432177201083239, 'weightPrec': 0.7488074901774064, 'macroRec': 0.6600512597266295, 'microRec': 0.7432177201083239, 'weightRec': 0.7432177201083239, 'macroF1': 0.5879440408733589, 'microF1': 0.7432177201083239, 'weightF1': 0.7420993987726012, 'subsetAcc': 0.7432177201083239, 'IoU': 0.43430327083335263}


  0%|          | 3/2487 [00:00<01:35, 26.11it/s]

Confusion Matrix:  
                     Bulkhead Or Sea Wall    Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall            1650307.0   346570.0   2975.0     6868.0  2006720.0   0.82
Rip Rap                          613497.0  1246990.0   6236.0    16882.0  1883605.0   0.66
Groin                               401.0     2532.0   5728.0     1006.0     9667.0   0.59
Breakwater                         1040.0     7388.0   2449.0    14028.0    24905.0   0.56
Sum                             2265245.0  1603480.0  17388.0    38784.0       mPA:   0.66
IoU                                  0.63       0.56     0.27       0.28      mIoU:   0.43
Precision                            0.73       0.78     0.33       0.36        OA:   0.74
F1-score                             0.77       0.72     0.42       0.44  F1-macro:   0.59


100%|██████████| 2487/2487 [01:22<00:00, 30.27it/s]
